In [0]:
#Find out hospital which serve most number of patients
from pyspark.sql import Window
from pyspark.sql.functions import rank, dense_rank,col

In [0]:
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key", "AK")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "SK")

In [0]:
Patient_records = spark.read.options(header=True, inferSchema=True, delimiter = ',').csv("s3://capstone-kanchan/healthcare/Patient_records.csv")
hospital = spark.read.options(header=True, inferSchema=True, delimiter = ',').csv("s3://capstone-kanchan/healthcare/hospital.csv")

In [0]:
Patient_records = Patient_records.dropDuplicates()
Patient_records_grouped = Patient_records.groupBy("hospital_id").count()

Patient_hospital = hospital.join(Patient_records_grouped, Patient_records_grouped.hospital_id == hospital.Hospital_id, "inner")
Patient_hospital = Patient_hospital.select(hospital.Hospital_id,"Hospital_name","count").orderBy("count", ascending=False)
Patient_hospital = Patient_hospital.withColumnRenamed("count", "PatientPerHospital")

Patient_hospital = Patient_hospital.withColumn("rank",dense_rank().over(Window.partitionBy().orderBy(col("PatientPerHospital").desc())))
Patient_hospital = Patient_hospital.filter(Patient_hospital.rank ==1)
Patient_hospital = Patient_hospital.select("Hospital_id","Hospital_name","PatientPerHospital")

In [0]:
Patient_hospital.write.format("redshift").option("url", "jdbc:redshift:/dev").\
   option("dbtable", "capstone.PatientPerHospital").\
   option("aws_iam_role", "arn:aws:iam/Redshift_admin").\
   option("tempdir", "s3a://capstone-kanchan/temp/").\
   option("user", "admin").\
   option("password", "Pw").save()